In [3]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [5]:
# enter details of your AML workspace
subscription_id = "606a443f-9d6b-4284-8781-9285eddbdbc3"
resource_group = "mlops_rg"
workspace = "sample_ml_workspace"

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [6]:
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="This is a new online endpoint",
    auth_mode="key",
)

print(endpoint_name)

endpt-04160226039492


In [7]:
model = Model(path="../artifacts/model/sklearn_regression_model.pkl")

In [8]:
env = Environment(
    conda_file="../artifacts/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

In [9]:
from pathlib import Path

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../artifacts/online_scoring",
        scoring_script="score.py",
    ),
    instance_type="Standard_F4S_V2",
    instance_count=1,
)

In [10]:
# Creating Local Endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Creating local endpoint (endpt-04160226039492) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-04160226039492', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/endpt-04160226039492'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f5806453940>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [11]:
# Create Local deployment
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Creating local deployment (endpt-04160226039492 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest
 ---> 83631454f85b
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 62e7f64fe3d8
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Running in f526cd2c2c8b
 ---> 7d5e9a4a881f
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> 572ae69a2371
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Running in 88315a4c2dd8
.Retrieving notices: - \ done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.1
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



python-3.9.22        | 22.5 MB   |            |   0% 
scipy-1.10.1         | 14.4 MB   |            |   0% 

scikit-learn-1.2.2   | 7.3 MB    |            |   0% 


n

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-04160226039492', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mksense_v1/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f580646a080>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '7713d7a5680d37a33a7ac52530aec294', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mksense_v1/notebooks'), 'creation_context': Non

In [12]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32770/score', 'openapi_uri': None, 'name': 'endpt-04160226039492', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mksense_v1/notebooks'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f58064ba470>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [15]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../sample-request.json",
    local=True,
)

'[11055.977245525679, 4503.079536107787, 12266.841758291512]'

In [16]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpt-04160226039492.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpt-04160226039492.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpt-04160226039492', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {'createdBy': 'Tarun Kumar', 'createdAt': '2025-04-16T02:49:44.334687+0000', 'lastModifiedAt': '2025-04-16T02:49:44.334687+0000', 'azureml.onlineendpointid': '/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/resourcegroups/mlops_rg/providers/microsoft.machinelearningservices/workspaces/sample_ml_workspace/onlineendpoints/endpt-04160226039492', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:268f9189-0238-4883-bf0e-df0de49c18b4:94e990ae-91a3-464f-9fdf-8a78e46598ae?api-versio

In [17]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpt-04160226039492 exists
Uploading online_scoring (0.0 MBs): 100%|██████████| 2826/2826 [00:00<00:00, 18734.10it/s]




..............................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-04160226039492', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/odidp:268f9189-0238-4883-bf0e-df0de49c18b4:90a509fa-7fe5-4b88-84ce-3ec10010456d?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/resourceGroups/mlops_rg/providers/Microsoft.MachineLearningServices/workspaces/sample_ml_workspace/onlineEndpoints/endpt-04160226039492/deployments/blue', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/tarunkw041/code/Users/tarunkw04/mksense_v1/notebooks', 'creation_context': <azure.ai.ml._restclient.v2023_04_01_preview.model

In [18]:
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpt-04160226039492.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpt-04160226039492.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpt-04160226039492', 'description': 'This is a new online endpoint', 'tags': {}, 'properties': {'createdBy': 'Tarun Kumar', 'createdAt': '2025-04-16T02:49:44.334687+0000', 'lastModifiedAt': '2025-04-16T02:58:17.846226+0000', 'azureml.onlineendpointid': '/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/resourcegroups/mlops_rg/providers/microsoft.machinelearningservices/workspaces/sample_ml_workspace/onlineendpoints/endpt-04160226039492', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/606a443f-9d6b-4284-8781-9285eddbdbc3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:268f9189-0238-4883-bf0e-df0de49c18b4:7cba03ee-3555-4482-ac21-9669ac8c207d?api-versio

---
## Inferencing

In [19]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="../sample-request.json",
)

'[11055.977245525679, 4503.079536107787, 12266.841758291512]'